https://github.com/fran-martinez/bio_ner_bert

https://github.com/MeRajat/SolvingAlmostAnythingWithBert/tree/master/biobert_ner

In [1]:
from transformers import pipeline

text = "Mouse thymus was used as a source of glucocorticoid receptor from normal CS lymphocytes."

nlp_ner = pipeline("ner",
                   model='fran-martinez/scibert_scivocab_cased_ner_jnlpba',
                   tokenizer='fran-martinez/scibert_scivocab_cased_ner_jnlpba')

nlp_ner(text)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'word': 'glucocorticoid',
  'score': 0.9894881248474121,
  'entity': 'B-protein',
  'index': 9},
 {'word': 'receptor',
  'score': 0.989505410194397,
  'entity': 'I-protein',
  'index': 10},
 {'word': 'normal',
  'score': 0.7680374383926392,
  'entity': 'B-cell_type',
  'index': 12},
 {'word': 'cs',
  'score': 0.5176804065704346,
  'entity': 'I-cell_type',
  'index': 13},
 {'word': 'lymphocytes',
  'score': 0.9898492097854614,
  'entity': 'I-cell_type',
  'index': 14}]

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Example
text = "Mouse thymus was used as a source of glucocorticoid receptor from normal CS lymphocytes."

# Load model
tokenizer = AutoTokenizer.from_pretrained("fran-martinez/scibert_scivocab_cased_ner_jnlpba")
model = AutoModelForTokenClassification.from_pretrained("fran-martinez/scibert_scivocab_cased_ner_jnlpba")

# Get input for BERT
input_ids = torch.tensor(tokenizer.encode(text)).unsqueeze(0)

# Predict
with torch.no_grad():
    outputs = model(input_ids)

# From the output let's take the first element of the tuple.
# Then, let's get rid of [CLS] and [SEP] tokens (first and last)
predictions = outputs[0].argmax(axis=-1)[0][1:-1]

# Map label class indexes to string labels.
for token, pred in zip(tokenizer.tokenize(text), predictions):
    print(token, '->', model.config.id2label[pred.numpy().item()])

mouse -> O
thymus -> O
was -> O
used -> O
as -> O
a -> O
source -> O
of -> O
glucocorticoid -> B-protein
receptor -> I-protein
from -> O
normal -> B-cell_type
cs -> I-cell_type
lymphocytes -> I-cell_type
. -> O


In [3]:
from transformers import pipeline

text = "Mouse thymus was used as a source of glucocorticoid receptor from normal CS lymphocytes."

nlp_ner = pipeline("ner",
                   model='gsarti/biobert-nli',
                   tokenizer='gsarti/biobert-nli')

nlp_ner(text)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at gsarti/biobert-nli and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'word': '[CLS]',
  'score': 0.6273071765899658,
  'entity': 'LABEL_1',
  'index': 0},
 {'word': 'Mouse',
  'score': 0.5842256546020508,
  'entity': 'LABEL_1',
  'index': 1},
 {'word': 'thy', 'score': 0.5531447529792786, 'entity': 'LABEL_1', 'index': 2},
 {'word': '##mus',
  'score': 0.5514998435974121,
  'entity': 'LABEL_1',
  'index': 3},
 {'word': 'was', 'score': 0.5502050518989563, 'entity': 'LABEL_0', 'index': 4},
 {'word': 'used',
  'score': 0.5568333268165588,
  'entity': 'LABEL_0',
  'index': 5},
 {'word': 'as', 'score': 0.5459309220314026, 'entity': 'LABEL_0', 'index': 6},
 {'word': 'a', 'score': 0.5106245279312134, 'entity': 'LABEL_1', 'index': 7},
 {'word': 'source',
  'score': 0.5343674421310425,
  'entity': 'LABEL_0',
  'index': 8},
 {'word': 'of', 'score': 0.5214571356773376, 'entity': 'LABEL_1', 'index': 9},
 {'word': 'g', 'score': 0.6315820813179016, 'entity': 'LABEL_1', 'index': 10},
 {'word': '##lu',
  'score': 0.5991693139076233,
  'entity': 'LABEL_1',
  'index': 11

In [5]:
# from transformers import pipeline

# text = "Mouse thymus was used as a source of glucocorticoid receptor from normal CS lymphocytes."

# nlp_ner = pipeline("ner",
#                    model='dmis-lab/biobert-v1.1',
#                    tokenizer='dmis-lab/biobert-v1.1')

# nlp_ner(text)

In [ ]:
# from transformers import pipeline

# text = "Mouse thymus was used as a source of glucocorticoid receptor from normal CS lymphocytes."

# nlp_ner = pipeline("ner",
#                    model='ktrapeznikov/biobert_v1.1_pubmed_squad_v2',
#                    tokenizer='ktrapeznikov/biobert_v1.1_pubmed_squad_v2')

# nlp_ner(text)

In [6]:
import random

from torch.optim import Adam
from torch.utils.data import DataLoader
from transformers import (get_linear_schedule_with_warmup,
                          BertForTokenClassification,
                          AutoTokenizer)



In [7]:
from collections import defaultdict
from dataclasses import dataclass, asdict
from typing import List, Optional, Dict, Tuple, Union

import numpy as np
import torch
from torch import nn
from torch.utils.data.dataset import Dataset
from transformers import PreTrainedTokenizer


@dataclass
class DataSample:
    """
    A single training/test example (sentence) for token classification.
    """
    words: List[str]
    labels: List[str]


@dataclass
class InputBert:
    """
    A single set of features of data.
    Property names are the same names as the corresponding inputs to a BERT model.
    """
    input_ids: torch.tensor
    attention_mask: torch.tensor
    token_type_ids: torch.tensor
    labels: Optional[torch.tensor] = None


class NerDataset(Dataset):
    def __init__(self,
                 dataset: List[DataSample],
                 tokenizer: PreTrainedTokenizer,
                 labels2ind: Dict[str, int],
                 max_len_seq: int = 512,
                 bert_hugging: bool = True):
        """
        Class that builds a torch Dataset specially designed for NER data.
        Args:
            dataset (list of `DataSample` instances): Each data sample is a dataclass
                that contains two fields: `words` and `labels`. Both are lists of `str`.
            tokenizer (`PreTrainedTokenizer`): Pre-trained tokenizer from transformers
                library. Usually loaded as `AutoTokenizer.from_pretrained(...)`.
            labels2ind (`dict`): maps `str` class labels into `int` indexes.
            max_len_seq (`int`): Max length sequence for each example (sentence).
            bert_hugging (`bool`):
        """
        super(NerDataset).__init__()
        self.bert_hugging = bert_hugging
        self.max_len_seq = max_len_seq
        self.label2ind = labels2ind
        self.features = data2tensors(data=dataset,
                                     tokenizer=tokenizer,
                                     label2idx=self.label2ind,
                                     max_seq_len=max_len_seq,
                                     pad_token_label_id=nn.CrossEntropyLoss().ignore_index)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, i) -> Union[Dict[str, torch.tensor],
                                      Tuple[List[torch.tensor], torch.tensor]]:
        if self.bert_hugging:
            return asdict(self.features[i])
        else:
            inputs = asdict(self.features[i])
            labels = inputs.pop('labels')
            return list(inputs.values()), labels


def get_labels(data: List[DataSample]) -> Tuple[Dict[str, int], Dict[str, int]]:
    """
    Automatically extract labels types from the data and its count.
    Args:
        data (list of `DataSample`): Each data sample is a dataclass that contains
            two fields: `words` and `labels`. Both are lists of `str`.
    Returns:
        labels2idx (`dict`): maps `str` class labels into `int` indexes.
        labels_count(`dict`): The number of words for each class label that appears in
            the dataset. Usufull information if you want to apply class weights on
            imbalanced data.
    """
    labels = set()
    labels_counts = defaultdict(int)
    for sent in data:
        labels.update(sent.labels)

        for label_ in sent.labels:
            labels_counts[label_] += 1

    if "O" not in labels:
        labels.add('O')
        labels_counts['0'] = 0

    # Convert list of labels ind a mapping labels -> index
    labels2idx = {label_: i for i, label_ in enumerate(labels)}
    return labels2idx, dict(labels_counts)


def get_class_weight_tensor(labels2ind: Dict[str, int],
                            labels_count: Dict[str, int]) -> torch.Tensor:
    """
    Get the class weights based on the class labels frequency within the dataset.
    Args:
        labels2ind (`dict`): maps `str` class labels into `int` indexes.
        labels_count (`dict`): The number of words for each class label that appears in
            the dataset.
    Returns:
        torch.Tensor with the class weights. Size (num_classes).
    """
    label2ind_list = [(k, v) for k, v in labels2ind.items()]
    label2ind_list.sort(key=lambda x: x[1])
    total_labels = sum([count for label, count in labels_count.items()])
    class_weights = [total_labels/labels_count[label] for label, _ in label2ind_list]
    return torch.tensor(np.array(class_weights)/max(class_weights), dtype=torch.float32)


def read_data_from_file(file_path: str, sep: str = '\t') -> List[DataSample]:
    """
    Load data from a txt file (BIO tagging format) and transform it into the
    required format (list of `DataSample` instances).
    Args:
        file_path (`str`): complete path where the data is located (path + filename).
        sep (`str`): Symbol used to separete word from label at each line. Default `\t`.
    Returns:
        List of `DataSample` instances containing words and labels.
    """
    examples = []
    words = []
    labels = []
    with open(file_path, encoding="utf-8") as f:
        for line in f:
            splits = line.split(sep)
            if len(splits) > 1:
                words.append(splits[0])
                labels.append(splits[-1].replace('\n', ''))
            else:
                examples.append(DataSample(words=words, labels=labels))
                words = []
                labels = []
    return examples


def data2tensors(data: List[DataSample],
                 tokenizer: PreTrainedTokenizer,
                 label2idx: Dict[str, int],
                 pad_token_label_id: int = -100,
                 max_seq_len: int = 512) -> List[InputBert]:
    """
    Takes data and converts it into tensors to feed the neural network.
    Args:
        data (`list`): List of `DataSample` instances containing words and labels.
        tokenizer (`PreTrainedTokenizer`): Pre-trained tokenizer from transformers
            library. Usually loaded as `AutoTokenizer.from_pretrained(...)`.
        label2idx (`dict`): maps `str` class labels into `int` indexes.
        pad_token_label_id (`int`): index to define the special token [PAD]
        max_seq_len (`int`): Max sequence length.
    Returns:
        List of `InputBert` instances. `InputBert` is a dataclass that contains
        `input_ids`, `attention_mask`, `token_type_ids` and `labels` (Optional).
    """

    features = []
    for sentence in data:
        tokens = []
        label_ids = []
        for word, label in zip(sentence.words, sentence.labels):
            subword_tokens = tokenizer.tokenize(text=word)

            # BERT could return an empty list of subtokens
            if len(subword_tokens) > 0:
                tokens.extend(subword_tokens)

                # Use the real label id for the first token of the word, and padding ids for the remaining tokens
                label_ids.extend([label2idx[label]] + [pad_token_label_id] * (len(subword_tokens) - 1))
                # if label.startswith('B'):
                #     label_ids.extend([label2idx[label]] + [label2idx[f"I{label[1:]}"]] * (len(subword_tokens) - 1))
                # else:
                #     label_ids.extend([label2idx[label]] + [label2idx[label]] * (len(subword_tokens) - 1))

        # Drop part of the sequence longer than max_seq_len (account also for [CLS] and [SEP])
        if len(tokens) > max_seq_len - 2:
            tokens = tokens[:max_seq_len - 2]
            label_ids = label_ids[: max_seq_len - 2]

        # Add special tokens  for the list of tokens and its corresponding labels.
        # For BERT: cls_token = '[CLS]' and sep_token = '[SEP]'
        # For RoBERTa: cls_token = '<s>' and sep_token = '</s>'
        tokens = [tokenizer.cls_token] + tokens + [tokenizer.sep_token]
        label_ids = [pad_token_label_id] + label_ids + [pad_token_label_id]

        # Create an attention mask (used to locate the padding)
        padding_len = (max_seq_len - len(tokens))
        attention_mask = [1] * len(tokens) + [0] * padding_len

        # Add padding
        tokens += [tokenizer.pad_token] * padding_len
        label_ids += [pad_token_label_id] * padding_len

        # Convert tokens to ids
        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # Create segment_id. All zeros since we only have one sentence
        segment_ids = [0] * max_seq_len

        # Assert all the input has the expected length
        assert len(input_ids) == max_seq_len
        assert len(label_ids) == max_seq_len
        assert len(attention_mask) == max_seq_len
        assert len(segment_ids) == max_seq_len

        # Append input features for each sequence/sentence
        features.append((InputBert(input_ids=torch.tensor(input_ids),
                                   attention_mask=torch.tensor(attention_mask),
                                   token_type_ids=torch.tensor(segment_ids),
                                   labels=torch.tensor(label_ids))))
    return features

In [8]:
from transformers import PreTrainedModel
import torch
from typing import Union


def get_optimizer_with_weight_decay(model: PreTrainedModel,
                                    optimizer: torch.optim.Optimizer,
                                    learning_rate: Union[float, int],
                                    weight_decay: Union[float, int]) -> torch.optim.Optimizer:
    """
    Apply weight decay to all the network parameters but those called `bias` or  `LayerNorm.weight`.
    Args:
        model (`PreTrainedModel`): model to apply weight decay.
        optimizer (`torch.optim.Optimizer`): The optimizer to use during training.
        learning_rate (`float` or `int`): value of the learning rate to use during training.
        weight_decay (`float` or `int`): value of the weight decay to apply.
    Returns:
        optimizer (`torch.optim.Optimizer`): the optimizer instantiated with the selected
        learning rate and the parameters with and without weight decay.
    """
    no_decay = ["bias", "LayerNorm.weight"]
    params = [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)]
    params_nd = [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)]
    optimizer_grouped_parameters = [{"params": params, "weight_decay": weight_decay},
                                    {"params": params_nd, "weight_decay": 0.0}]

    return optimizer(optimizer_grouped_parameters, lr=learning_rate)

In [10]:
!pip install seqeval

  Created wheel for seqeval: filename=seqeval-0.0.12-py3-none-any.whl size=7423 sha256=fbc0723be9f04fb8dd00c6094dab0b3aedcf39ecbe9724fb38eeac54f31cc915
  Stored in directory: /Users/sdeshpande/Library/Caches/pip/wheels/04/bf/20/90daf50b5a8173fc6ee715e6ebb0a16202cc43df471e323e88
Successfully built seqeval


In [11]:
import itertools
import os
from typing import List, Tuple, Union, Dict, Optional

import numpy as np
import torch
from matplotlib import pyplot as plt
from seqeval.metrics import classification_report as seqeval_report
from seqeval.metrics import f1_score
from sklearn.metrics import classification_report as sklearn_report
from torch import nn
from torch.nn.modules.loss import CrossEntropyLoss
from torch.utils.data.dataloader import DataLoader
from transformers import PreTrainedModel, PreTrainedTokenizer


class BertTrainer:
    def __init__(self,
                 model: PreTrainedModel,
                 tokenizer: PreTrainedTokenizer,
                 optimizer: torch.optim.Optimizer,
                 n_epochs: int,
                 labels2ind: Dict[str, int],
                 scheduler: Optional[torch.optim.lr_scheduler.LambdaLR] = None,
                 device: str = 'cpu',
                 clipping: Optional[Union[int, float]] = None,
                 accumulate_grad_every: int = 1,
                 print_every: int = 10,
                 print_val_mistakes: bool = False,
                 output_dir: str = './'):

        """
        Complete training and evaluation loop in Pytorch specially designed for
        BERT-based models from transformers library. It allows to save the model
        from the epoch with the best F1-score and the tokenizer. The class
        optionally generates reports and figures with the obtained results that
        are automatically stored in disk.
        Args:
            model (`PreTrainedModel`): Pre-trained model from transformers library.
                For NER, usually loaded as `BertForTokenClassification.from_pretrained(...)`
            tokenizer (`PreTrainedTokenizer`): Pre-trained tokenizer from transformers library.
                Usually loaded as `AutoTokenizer.from_pretrained(...)`
            optimizer (`torch.optim.Optimizer`): Pytorch Optimizer
            n_epochs (`int`): Number of epochs to train.
            labels2ind (`dict`): maps `str` class labels into `int` indexes.
            scheduler (`torch.optim.lr_scheduler.LambdaLR`, `Optional`): Pytorch scheduler. It sets a
                different learning rate for each training step to update the network weights.
            device (`str`): Type of device where to train the network. It must be `cpu` or `cuda`.
            clipping (`int` or `float`, `Optional`): max norm to apply to the gradients. If None,
                no graddient clipping is applied.
            accumulate_grad_every (`int`): How often you want to accumulate the gradient. This is useful
                when there are limitations in the batch size due to memory issues. Let's say that in your
                GPU only fits a model with batch size of 8 and you want to try a batch size of 32. Then,
                you should set this parameter to 4 (8*4=32). Internally, a loop will be ran 4 times
                accumulating the gradient for each step. Later, the network parameters will be updated.
                So at the end, this is equivalent to train your network with a batch size of 32. The batch
                size is inferred from `dataloader_train` argument.
            print_every (`int`): How often you want to print loss. Measured in batches where a batch is
                considered batch_size * accumulate_grad_every.
            print_val_mistakes (`bool`): whether to print validation examples (sentences) where the model
                commits at least one mistake. It is printed after each epoch. The printed info is the word
                within each sentence, its predicted label and the real label. This is very useful to
                inspect the behaviour of your model.
            output_dir (`str`): Directory where file reports and images are saved.
        Methods:
            train(dataloader_train: DataLoader, dataloader_val: Optional[DataLoader] = None)
                Complete training and evaluation (optional) loop in Pytorch.
            evaluate(dataloader_val: DataLoader, epoch: int = 0, verbose: bool = False)
                Evaluation on test data.
        """

        self.tokenizer = tokenizer
        self.model = model
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.n_epochs = n_epochs
        self.labels2ind = labels2ind
        self.inds2labels = {v: k for k, v in self.labels2ind.items()}
        self.device = device
        self.clipping = clipping
        self.accumulate_grad_every = accumulate_grad_every
        self.print_every = print_every
        self.print_val_mistakes = print_val_mistakes
        self.output_dir = output_dir

        os.makedirs(self.output_dir, exist_ok=True)

    def _reformat_predictions(self,
                              y_true: List[List[int]],
                              y_pred: List[List[int]],
                              input_ids: List[List[str]]
                              ) -> Tuple[List[List[str]],
                                         List[List[str]],
                                         List[List[str]]]:
        """
        Takes batch of tokens, labels (class indexes) and predictions (class indexes)
        and get rid of unwanted tokens, that is, those that have as label the index
        to ignore (i.e. padding tokens).  It also converts the label and prediction
        indexes into their corresponding class name.
        Args:
            y_true (list of lists `int`): indexes of the real labels
            y_pred (list of lists `int`): indexes of the predicted classes
            input_ids (list of lists `str`) : tokens
        Returns:
            Tuple that contains the transformed input arguments
        """
        # Map indexes to labels and remove ignored indexes
        true_list = [[] for _ in range(len(y_true))]
        pred_list = [[] for _ in range(len(y_pred))]
        input_list = [[] for _ in range(len(input_ids))]

        for i in range(len(y_true)):
            for j in range(len(y_true[0])):
                if y_true[i][j] != CrossEntropyLoss().ignore_index:
                    true_list[i].append(self.inds2labels[y_true[i][j]])
                    pred_list[i].append(self.inds2labels[y_pred[i][j]])
                    input_list[i].append(input_ids[i][j])

        return true_list, pred_list, input_list

    def _print_missclassified_val_examples(self,
                                           y_true: List[List[str]],
                                           y_pred: List[List[str]],
                                           input_ids: List[List[str]]):
        """
        print validation examples (sentences) where the model commits at least
        one mistake. It is printed after each epoch. This is very useful to
        inspect the behaviour of your model.
        Args:
            y_true (list of lists `str`): real labels
            y_pred (list of lists `str`): predicted classes
            input_ids (list of lists `str`) : tokens
        Examples::
                TOKEN          LABEL          PRED
                immunostaining O              O
                showed         O              O
                the            O              O
                estrogen       B-cell_type    B-cell_type
                receptor       I-cell_type    I-cell_type
                cells          I-cell_type    O
                                  ·
                                  ·
                                  ·
                synovial       O              O
                tissues        O              O
                .              O              O
        """
        # Print some examples (where the model fails)
        for i in range(len(input_ids)):
            if y_true[i] != y_pred[i]:
                tokens = self.tokenizer.convert_ids_to_tokens(input_ids[i])
                max_len_token = max([len(t) for t in tokens] +
                                    [len(la) for la in self.labels2ind.keys()])

                print(f"\n{'TOKEN':<{max_len_token}}",
                      f"{'LABEL':<{max_len_token}}",
                      f"{'PRED':<{max_len_token}}")

                for token, label_true, label_pred in zip(tokens, y_true[i], y_pred[i]):
                    print(f"{token:<{max_len_token}}",
                          f"{label_true:<{max_len_token}}",
                          f"{label_pred:<{max_len_token}}")

    def _write_report_to_file(self,
                              report_entities: str,
                              report_tokens: str,
                              epoch: int,
                              tr_loss: float,
                              val_loss: float):
        """
        Writes and saves the following info into a file called `classification_report.txt`
        within the directory `output_dir` for the model from the best epoch:
            - Classification report at span/entity level (for validation dataset).
            - Classification report at word level (for validation dataset).
            - Epoch where the best model was found (best F1-score in validation dataset)
            - Training loss from the best epoch.
            - Validation loss from the best epoch.
        Args:
            report_entities (`str`): classification report at entity/span level.
            report_tokens (`str`): classification report at word level.
            epoch (`int`): epoch
            tr_loss (`float`): training loss
            val_loss (`float`): validation loss
        Returns:
        """
        with open(os.path.join(self.output_dir, 'classification_report.txt'), 'w') as f:
            f.write(report_entities)
            f.write(f'\n{report_tokens}')
            f.write(f"\nEpoch: {epoch} "
                    f"\n- Training Loss: {tr_loss}"
                    f"\n- Validation Loss: {val_loss}")

    def _save_model(self):
        if not isinstance(self.model, PreTrainedModel):
            raise ValueError("Trainer.model appears to not be a PreTrainedModel")
        self.model.save_pretrained(self.output_dir)
        self.tokenizer.save_pretrained(self.output_dir)

    def _estimate_gradients(self, batch: Dict[str, torch.Tensor]) -> float:
        # Send tensors to device
        batch = {k: v.to(self.device) for k, v in batch.items()}

        # estimate loss and gradient
        loss, _ = self.model(**batch)
        loss.backward()

        return loss.item()

    def _update_network_params(self):
        # Graddient clipping
        if self.clipping is not None:
            nn.utils.clip_grad_norm_(self.model.parameters(), self.clipping)

        # Udate parameters (accumulated gradient based on accumulated grad)
        self.optimizer.step()
        if self.scheduler is not None:
            self.scheduler.step()
        self.model.zero_grad()

    def _validation_step(self,
                         batch: Dict[str, torch.Tensor]
                         ) -> Tuple[float, np.ndarray]:
        # Send tensors to device
        batch = {k: v.to(self.device) for k, v in batch.items()}

        # Predict and estimate error
        with torch.no_grad():
            loss, pred = self.model(**batch)

        return loss.item(), pred.detach().cpu().numpy()

    def evaluate(self,
                 dataloader_val: DataLoader,
                 epoch: int = 0,
                 verbose: bool = False) -> Tuple[float, float, str, str]:
        """
        Args:
            dataloader_val:
            epoch:
            verbose:
        Returns:
        """
        n_steps_val = len(dataloader_val)
        self.model.eval()

        val_loss_cum = .0
        y_pred = []
        y_true = []
        input_ids = []
        for step, batch in enumerate(dataloader_val):
            val_loss, pred = self._validation_step(batch)
            val_loss_cum += val_loss
            y_true.extend(batch['labels'].tolist())
            y_pred.extend(pred.argmax(axis=-1).tolist())
            input_ids.extend(batch['input_ids'].tolist())

        y_true, y_pred, input_ids = self._reformat_predictions(y_true, y_pred, input_ids)

        # Performance Reports and loss
        report_entities = seqeval_report(y_true=y_true, y_pred=y_pred, digits=4)
        report_tokens = sklearn_report(y_true=list(itertools.chain(*y_true)),
                                       y_pred=list(itertools.chain(*y_pred)), digits=4)

        loss_val_epoch = val_loss_cum / n_steps_val

        if verbose:
            print(f"- Epoch: {epoch}/{self.n_epochs - 1} - Validation Loss: {loss_val_epoch}")
            print(report_entities)
            print(report_tokens)

        # Print some examples (where the model fails)
        if self.print_val_mistakes and verbose:
            self._print_missclassified_val_examples(y_true, y_pred, input_ids)

        # Save model and write report to txt file
        f1 = f1_score(y_true=y_true, y_pred=y_pred)

        return loss_val_epoch, f1, report_entities, report_tokens

    def train(self,
              dataloader_train: DataLoader,
              dataloader_val: Optional[DataLoader] = None
              ) -> Tuple[List[float], List[float]]:
        """
        Complete training and evaluation (optional) loop in Pytorch.
        Args:
            dataloader_train (`torch.utils.data.dataloader.DataLoader`): Pytorch dataloader.
            dataloader_val (`torch.utils.data.dataloader.DataLoader`, `Optional`):
                Pytorch dataloader. If `None` no validation will be performed.
        Returns:
            loss_tr_epochs (list of `float`): training loss for each epoch
            loss_val_epochs (list of `float`): validation loss for each epoch
        """
        loss_tr_epochs = []
        loss_val_epochs = []
        f1_best = .0
        lrs = []
        self.model.to(self.device)

        for epoch in range(self.n_epochs):
            tr_loss_mean = .0
            tr_loss_cum = .0
            step = -1

            # Training
            # -----------------------------
            self.model.train()
            self.model.zero_grad()
            for i, batch in enumerate(dataloader_train):
                # Estimate gradients and accumulate them
                tr_loss = self._estimate_gradients(batch)
                tr_loss_cum += tr_loss

                # Update params every acumulated steps
                if (i + 1) % self.accumulate_grad_every == 0:
                    self._update_network_params()
                    if self.scheduler is not None:
                        lrs.append(self.scheduler.get_last_lr()[0])
                    step += 1
                else:
                    continue

                if step % self.print_every == 0:
                    tr_loss_mean = tr_loss_cum/(i+1)
                    print(f"- Epoch: {epoch}/{self.n_epochs - 1}",
                          f"- Step: {step:3}/{(len(dataloader_train)// self.accumulate_grad_every) - 1}",
                          f"- Training Loss: {tr_loss_mean:.6f}")

            loss_tr_epochs.append(tr_loss_mean)
            print(f"- Epoch: {epoch}/{self.n_epochs - 1} - Training Loss: {tr_loss_mean}")

            # Plot training curve
            plt.plot(loss_tr_epochs)
            plt.xlabel('#Epochs')
            plt.ylabel('Error')
            plt.legend(['training'])

            # Validation
            # -----------------------------
            if dataloader_val is not None:
                val_loss, f1, report_ent, report_toks = self.evaluate(dataloader_val,
                                                                      epoch=epoch,
                                                                      verbose=True)
                loss_val_epochs.append(val_loss)

                if f1 > f1_best:
                    f1_best = f1
                    self._save_model()
                    self._write_report_to_file(report_ent, report_toks, epoch,
                                               tr_loss_mean, val_loss)

                # Plot val curve
                plt.plot(loss_val_epochs)
                plt.legend(['training', 'validation'])

            plt.tight_layout()
            plt.savefig(os.path.join(self.output_dir, 'error_curves.jpg'))
            plt.close()

            # Plot learning rate curve
            plt.plot(lrs)
            plt.xlabel('#Batches')
            plt.ylabel('Learning rate')
            plt.tight_layout()
            plt.savefig(os.path.join(self.output_dir, 'learning_rate.jpg'))
            plt.close()
        return loss_tr_epochs, loss_val_epochs

In [12]:
# https://github.com/cambridgeltl/MTL-Bioinformatics-2016/tree/master/data
DATA_TR_PATH = '/Users/sdeshpande/Desktop/bioinformatices/BIO_NER_data/data/BC5CDR-chem-IOB/train.tsv'
DATA_VAL_PATH = '/Users/sdeshpande/Desktop/bioinformatices/BIO_NER_data/data/BC5CDR-chem-IOB/devel.tsv'
DATA_TEST_PATH = None
SEED = 42

# MODEL
MODEL_NAME = 'allenai/scibert_scivocab_uncased'
MAX_LEN_SEQ = 128
DEVICE = 'cpu'

# Optimization parameters
N_EPOCHS = 2
BATCH_SIZE = 8
BATCH_SIZE_VAL = 28
WEIGHT_DECAY = 0
LEARNING_RATE = 1e-4  # 2e-4
RATIO_WARMUP_STEPS = .1
DROPOUT = .3
ACUMULATE_GRAD_EVERY = 4
OPTIMIZER = Adam

# Seeds
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [13]:
# get data
training_set = read_data_from_file(DATA_TR_PATH)
val_set = read_data_from_file(DATA_VAL_PATH)

# Automatically extract labels and their indexes from data.
labels2ind, labels_count = get_labels(training_set + val_set)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Create loaders for datasets
training_set = NerDataset(dataset=training_set,
                          tokenizer=tokenizer,
                          labels2ind=labels2ind,
                          max_len_seq=MAX_LEN_SEQ)

val_set = NerDataset(dataset=val_set,
                     tokenizer=tokenizer,
                     labels2ind=labels2ind,
                     max_len_seq=MAX_LEN_SEQ)

dataloader_tr = DataLoader(dataset=training_set,
                           batch_size=BATCH_SIZE,
                           shuffle=True)

dataloader_val = DataLoader(dataset=val_set,
                            batch_size=BATCH_SIZE_VAL,
                            shuffle=False)

# Load model
nerbert = BertForTokenClassification.from_pretrained(MODEL_NAME,
                                                     hidden_dropout_prob=DROPOUT,
                                                     attention_probs_dropout_prob=DROPOUT,
                                                     num_labels=len(labels2ind),
                                                     id2label={str(v): k for k, v in labels2ind.items()})

# Prepare optimizer and schedule (linear warmup and decay)
optimizer = get_optimizer_with_weight_decay(model=nerbert,
                                            optimizer=OPTIMIZER,
                                            learning_rate=LEARNING_RATE,
                                            weight_decay=WEIGHT_DECAY)

training_steps = (len(dataloader_tr)//ACUMULATE_GRAD_EVERY) * N_EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer=optimizer,
                                            num_warmup_steps=training_steps * RATIO_WARMUP_STEPS,
                                            num_training_steps=training_steps)

# Trainer
trainer = BertTrainer(model=nerbert,
                      tokenizer=tokenizer,
                      optimizer=optimizer,
                      scheduler=scheduler,
                      labels2ind=labels2ind,
                      device=DEVICE,
                      n_epochs=N_EPOCHS,
                      accumulate_grad_every=ACUMULATE_GRAD_EVERY,
                      output_dir='./trained_models')

# Train and validate model
trainer.train(dataloader_train=dataloader_tr,
              dataloader_val=dataloader_val)

# Test the model on test set if any
if DATA_TEST_PATH is not None:
    print(f"{'*'*40}\n\t\tEVALUATION ON TEST SET\n{'*'*40}")
    test_set = read_data_from_file(DATA_TEST_PATH)

    test_set = NerDataset(dataset=test_set,
                          tokenizer=tokenizer,
                          labels2ind=labels2ind,
                          max_len_seq=MAX_LEN_SEQ)

    dataloader_test = DataLoader(dataset=test_set,
                                 batch_size=BATCH_SIZE_VAL)

    trainer.evaluate(dataloader_test, verbose=True)

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initi

- Epoch: 0/1 - Step:   0/141 - Training Loss: 0.962044
- Epoch: 0/1 - Step:  10/141 - Training Loss: 0.547755
- Epoch: 0/1 - Step:  20/141 - Training Loss: 0.414147
- Epoch: 0/1 - Step:  30/141 - Training Loss: 0.321616
- Epoch: 0/1 - Step:  40/141 - Training Loss: 0.261613
- Epoch: 0/1 - Step:  50/141 - Training Loss: 0.221518
- Epoch: 0/1 - Step:  60/141 - Training Loss: 0.193794
- Epoch: 0/1 - Step:  70/141 - Training Loss: 0.173550
- Epoch: 0/1 - Step:  80/141 - Training Loss: 0.158017
- Epoch: 0/1 - Step:  90/141 - Training Loss: 0.145708
- Epoch: 0/1 - Step: 100/141 - Training Loss: 0.134616
- Epoch: 0/1 - Step: 110/141 - Training Loss: 0.125471
- Epoch: 0/1 - Step: 120/141 - Training Loss: 0.118035
- Epoch: 0/1 - Step: 130/141 - Training Loss: 0.111314
- Epoch: 0/1 - Step: 140/141 - Training Loss: 0.105312
- Epoch: 0/1 - Training Loss: 0.10531191656395714
- Epoch: 0/1 - Validation Loss: 0.03321884393801333
           precision    recall  f1-score   support

 Chemical     0.8944 

In [14]:
nlp_ner = pipeline("ner", model='./trained_models', tokenizer='./trained_models')

In [16]:
text = 'Triheptanoin (Dojolvi™), a synthetic medium-chain triglyceride, is being developed by Ultragenyx Pharmaceutical as a pharmaceutical-grade anaplerotic compound for use in the treatment of inherited metabolic disorders.'

In [17]:
nlp_ner(text)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'word': 'tri',
  'score': 0.9985841512680054,
  'entity': 'B-Chemical',
  'index': 1},
 {'word': '##hep',
  'score': 0.989841878414154,
  'entity': 'I-Chemical',
  'index': 2},
 {'word': '##tan',
  'score': 0.9938075542449951,
  'entity': 'I-Chemical',
  'index': 3},
 {'word': '##oin',
  'score': 0.9875107407569885,
  'entity': 'I-Chemical',
  'index': 4},
 {'word': '[UNK]',
  'score': 0.9598410725593567,
  'entity': 'B-Chemical',
  'index': 6}]

BC5CDR tags :-
    'B-Chemical', 
    'O', 
    'B-Disease', 
    'I-Disease', 
    'I-Chemical'

BioNLP13CG tags :-
'B-Anatomical_system',
'B-Cancer',
'B-Cell', 
'B-Cellular_component',
'B-Developing_anatomical_structure',
'B-Gene_or_gene_product', 
'B-Immaterial_anatomical_entity',
'B-Multi-tissue_structure',
'B-Organ',
'B-Organism', 
'B-Organism_subdivision',
'B-Organism_substance',
'B-Pathological_formation', 
'B-Simple_chemical',
'B-Tissue',
'I-Amino_acid',
'I-Anatomical_system',
'I-Cancer', 
'I-Cell',
'I-Cellular_component',
'I-Developing_anatomical_structure',
'I-Gene_or_gene_product', 
'I-Immaterial_anatomical_entity',
'I-Multi-tissue_structure',
'I-Organ',
'I-Organism', 
'I-Organism_subdivision',
'I-Organism_substance',
'I-Pathological_formation',
'I-Simple_chemical', 
'I-Tissue',
'O'